# Lista de Apps a Estudiar

Con sus respectivos appID necesarios para exportar comentarios u opiniones

In [45]:
#Imports 

import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time


# PASO 1

In [2]:
# Obtener la lista de todas las apps en Steam mediante su API
response = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")
data = response.json()

# Lista de juegos 
apps = data['applist']['apps']
appNames = []
appIds = []
# Imprimir los juegos
for app in apps:
    appNames.append(app['name'])
    appIds.append(app['appid'])

steamapps = pd.DataFrame({
    'Name': appNames,
    'AppID': appIds
})

In [3]:
steamapps

,Name,AppID
0,,1941401
1,,1897482
2,,2112761
3,,2016512
4,,1820332
...,...,...
208981,自由江湖,2444860
208982,Hunt the Pale Gods,2997060
208983,Duels of Fortune,2371130
208984,Daydream Mosaics 5: Crimson Veil,3117990


In [4]:
# Obtener la lista de los juegos de Steam 


# URL de la página de Steam 
url = "https://store.steampowered.com/search/?ignore_preferences=1&supportedlang=spanish&untags=12095&category1=998&filter=topsellers&ndl=1&page=1"
# Realizar la solicitud HTTP a la página
response = requests.get(url)
main = BeautifulSoup(response.content, 'html.parser')

# Lista para almacenar los datos
names = []

# Numero de paginas
total_pages = int(main.find('div', {'class': 'search_pagination_right'}).find_all('a')[-2].text)
#print(total_pages)

# Miramos todas las paginas ya que cada una contiene solo 25 titulos
for page in range(1, total_pages + 1):
    #contador = 0
    response = requests.get(url + '&page=' + str(page))

    if response.status_code != 200:
        print(f"Error: No se pudo acceder a la página. Código de estado: {response.status_code}")
    else:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Buscar los elementos de los juegos en la página
        games = soup.find_all('div', class_='responsive_search_name_combined')

        # Extraer nombres y precios de los juegos
        for game in games:
            names.append(game.find('span', {'class': 'title'}).text)
            #contador = contador + 1

        #print(contador, page)

# Crear un DataFrame de Pandas para organizar los datos
gamesbestsellers = pd.DataFrame({
    'Name': names,
})

# Mostrar las primeras filas del DataFrame
display(gamesbestsellers)


,Name
0,PUBG: BATTLEGROUNDS
1,Counter-Strike 2
2,Call of Duty®: Black Ops 6
3,God of War
4,EA SPORTS FC™ 25
...,...
1694,Possessed
1695,Fighters Unleashed
1696,Fireside
1697,UBERMOSH + Original Soundtrack


In [5]:
cruce = pd.merge(gamesbestsellers, steamapps, on='Name', how='left')
display(cruce)

,Name,AppID
0,PUBG: BATTLEGROUNDS,578080.0
1,Counter-Strike 2,730.0
2,Call of Duty®: Black Ops 6,2933620.0
3,God of War,1593500.0
4,EA SPORTS FC™ 25,3102940.0
...,...,...
1721,Possessed,607210.0
1722,Fighters Unleashed,566560.0
1723,Fireside,1722700.0
1724,UBERMOSH + Original Soundtrack,NaN


In [6]:
cruce_limpio = cruce.copy().drop_duplicates()
display(cruce_limpio)

,Name,AppID
0,PUBG: BATTLEGROUNDS,578080.0
1,Counter-Strike 2,730.0
2,Call of Duty®: Black Ops 6,2933620.0
3,God of War,1593500.0
4,EA SPORTS FC™ 25,3102940.0
...,...,...
1721,Possessed,607210.0
1722,Fighters Unleashed,566560.0
1723,Fireside,1722700.0
1724,UBERMOSH + Original Soundtrack,NaN


In [7]:
display(cruce_limpio[cruce_limpio['AppID'].isna()])

,Name,AppID
5,Call of Duty®: Black Ops III,NaN
33,Fallout 4: Game of the Year Edition,NaN
37,Apex Legends™,NaN
39,Call of Duty®: Black Ops II,NaN
43,The Elder Scrolls® Online,NaN
...,...,...
1672,Adam's Venture: Origins,NaN
1685,Slikoban,NaN
1686,X-COM: Complete Pack,NaN
1699,Perseverance Mission - Astronaut Charlie,NaN


In [8]:
games_id = cruce_limpio[cruce_limpio['AppID'].notna()].copy()
#al generarse NaN nos convierte los valores de ID a float y eso en el futuro nos daria problemas, asi que los convertimos en int otra vez
games_id['AppID'] = games_id['AppID'].astype('int')
display(games_id)

,Name,AppID
0,PUBG: BATTLEGROUNDS,578080
1,Counter-Strike 2,730
2,Call of Duty®: Black Ops 6,2933620
3,God of War,1593500
4,EA SPORTS FC™ 25,3102940
...,...,...
1720,Farming Tractor Simulator: Big Farm,2315010
1721,Possessed,607210
1722,Fighters Unleashed,566560
1723,Fireside,1722700


In [24]:
games_id.to_csv('games_id.csv', index=False)

Esta es la lista final con sus appID, vamos ahora a obtener datos de cada uno de los juegos

# PASO 2

Ejecutar desde aqui si se ha recogido bien el csv de games_id

In [ ]:
games_id = pd.read_csv('games_id.csv') 

## Ejemplo 

In [10]:
#Analizamos un ejemplo primero:
appid_test = 730
request_appdata_test = requests.get(f"https://store.steampowered.com/api/appdetails?appids={appid_test}")       
if request_appdata_test.status_code == 200:
    appdata_test = request_appdata_test.json()
    appdata_test = appdata_test[str(appid_test)] #Depuramos el primer bloque ya que lleva el appid como key principal del json y no nos interesa
appdata_test

{'success': True,
 'data': {'type': 'game',
  'name': 'Counter-Strike 2',
  'steam_appid': 730,
  'required_age': 0,
  'is_free': True,
  'dlc': [2678630],
  'detailed_description': 'For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.<br><br>A free upgrade to CS:GO, Counter-Strike 2 marks the largest technical leap in Counter-Strike’s history. Built on the Source 2 engine, Counter-Strike 2 is modernized with realistic physically-based rendering, state of the art networking, and upgraded Community Workshop tools.<br><br>In addition to the classic objective-focused gameplay that Counter-Strike pioneered in 1999, Counter-Strike 2 features:<br><br><ul class="bb_ul"><li>All-new CS Ratings with the updated Premier mode<br></li><li>Global and Regional leaderboards<br></li><li>Upgraded and overhauled maps<br></li><li>Game-c

In [11]:
#Creamos el diccionario
app_test = {}
app_test['Name'] = appdata_test['data']['name']
app_test['AppID'] = appdata_test['data']['steam_appid']
app_test['Description'] = appdata_test['data']['short_description']
app_test['Devs'] = [appdata_test['data']['developers']]
app_test['Publishers'] = [appdata_test['data']['publishers']]
app_test['Genres'] = [x['description'] for x in appdata_test['data']['genres']]
# app_test['Amount_Recommends'] = appdata_test['data']['recommendations']['total'] No se usa al final
app_test['Release_Date'] = appdata_test['data']['release_date']['date']
app_test['Soon'] = appdata_test['data']['release_date']['coming_soon']
app_test['Notes'] = appdata_test['data']['content_descriptors']['notes']
app_test['Success'] = appdata_test['success']
app_test

{'Name': 'Counter-Strike 2',
 'AppID': 730,
 'Description': 'For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.',
 'Devs': [['Valve']],
 'Publishers': [['Valve']],
 'Genres': ['Action', 'Free To Play'],
 'Release_Date': '21 Aug, 2012',
 'Soon': False,
 'Notes': 'Includes intense violence and blood.',
 'Success': True}

In [12]:
#Vemos uno que no llega a analizar para ver como hacer la excepcion 
appid_test_2 = 3102940
request_appdata_test_2 = requests.get(f"https://store.steampowered.com/api/appdetails?appids={appid_test_2}")       
if request_appdata_test_2.status_code == 200:
    appdata_test_2 = request_appdata_test_2.json()
    appdata_test_2 = appdata_test_2[str(appid_test_2)] #Depuramos el primer bloque ya que lleva el appid como key principal del json y no nos interesa
appdata_test_2

{'success': False}

## Funciones para esta parte

In [40]:
# Definimos una funcion de test por si acaso, para leer JSONs y ver si falla alguno en concreto
def test(appid_test):
    request_appdata_test = requests.get(f"https://store.steampowered.com/api/appdetails?appids={appid_test}")       
    if request_appdata_test.status_code == 200:
        appdata_test = request_appdata_test.json()
        appdata_test = appdata_test[str(appid_test)] #Depuramos el primer bloque ya que lleva el appid como key principal del json y no nos interesa
    return appdata_test

#Funcion para recoger los datos que nos interesan del json y usarlos en un DF
def appdata_req(game):
    request_appdata = requests.get(f"https://store.steampowered.com/api/appdetails?appids={game['AppID']}")  
    app = {}
    if request_appdata.status_code == 200:
        appdata = request_appdata.json()
        appdata = appdata[str(game['AppID'])] #Depuramos el primer bloque ya que lleva el appid como key principal del json y no nos interesa
        if appdata['success'] == True:
            app['Name'] = appdata.get('data',None).get('name',None)
            app['AppID'] = appdata.get('data',None).get('steam_appid',None)
            app['Description'] = appdata.get('data',None).get('short_description',None)
            app['Devs'] = [appdata.get('data',None).get('developers',None)]
            app['Publishers'] = [appdata.get('data',None).get('publishers',None)]
            if appdata.get('data',None).get('genres',None) != None:
                app['Genres'] = [[x.get('description',None) for x in appdata.get('data',None).get('genres',None)]] # Hay que depurar este más aún
            else: app['Genres'] = None
            app['Release_Date'] = appdata.get('data',None).get('release_date',None).get('date',None)
            app['Soon'] = appdata.get('data',None).get('release_date',None).get('coming_soon',None)
            app['Notes'] = appdata.get('data',None).get('content_descriptors',None).get('notes',None)
            app['Success'] = appdata.get('success',None)
        else: 
            app['Name'] = [game['Name']]
            app['AppID'] = [game['AppID']]
            app['Description'] = None
            app['Devs'] = None
            app['Publishers'] = None
            app['Genres'] = None
            app['Release_Date'] = None
            app['Soon'] = None
            app['Notes'] = None
            app['Success'] = appdata['success']
        
    return app


## Particionamos y vamos comprobando uno a uno 

Estoy capado a 200 request cada 5mins, asi que carga rara la info

In [115]:
games_id_details_1 = pd.DataFrame()
for _,game in games_id[0:150].iterrows():
    dict = appdata_req(game)
    games_id_details_1 = pd.concat([games_id_details_1, pd.DataFrame(dict)], ignore_index = True)

games_id_details_1

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,PUBG: BATTLEGROUNDS,578080,Play PUBG: BATTLEGROUNDS for free. Land on str...,"[KRAFTON, Inc.]","[KRAFTON, Inc.]","[Action, Adventure, Massively Multiplayer, Fre...","21 Dec, 2017",False,None,True
1,Counter-Strike 2,730,"For over two decades, Counter-Strike has offer...",[Valve],[Valve],"[Action, Free To Play]","Aug 21, 2012",False,Includes intense violence and blood.,True
2,Call of Duty®: Black Ops 6,2933620,Call of Duty®: Black Ops 6 is signature Black ...,"[Treyarch, Raven Software, Beenox, High Moon S...",[Activision],[Action],"24 Oct, 2024",True,None,True
3,God of War,1593500,His vengeance against the Gods of Olympus year...,"[Santa Monica Studio, Jetpack Interactive]",[PlayStation Publishing LLC],"[Action, Adventure, RPG]","14 Jan, 2022",False,Gameplay consists of frequent combat scenarios...,True
4,EA SPORTS FC™ 25,3102940,None,None,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...
145,Stumble Guys,1677740,Race through obstacle courses against up to 32...,[Scopely],[Scopely],"[Action, Casual, Indie, Massively Multiplayer,...","7 Oct, 2021",False,None,True
146,Stellaris,281990,Explore a galaxy full of wonders in this sci-f...,[Paradox Development Studio],[Paradox Interactive],"[Simulation, Strategy]","9 May, 2016",False,None,True
147,Governor of Poker 3,436150,Governor of Poker 3 is the best multiplayer po...,[Playtika Ltd.],[Playtika Ltd.],"[Casual, Massively Multiplayer, Simulation, Fr...","19 Feb, 2016",False,None,True
148,Brawlhalla,291550,An epic platform fighter for up to 8 players o...,[Blue Mammoth Games],[Ubisoft],"[Action, Indie, Free To Play]","17 Oct, 2017",False,None,True


In [36]:
time.sleep(300)
games_id_details_2 = pd.DataFrame()
for _,game in games_id[150:300].iterrows():
    dict = appdata_req(game)
    games_id_details_2 = pd.concat([games_id_details_2, pd.DataFrame(dict)], ignore_index = True)

games_id_details_2

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Persona 3 Reload,2161700,Dive into the Dark Hour and awaken the depths ...,[ATLUS],[SEGA],"[Adventure, RPG, Strategy]","1 Feb, 2024",False,"This game includes content such as blood, part...",True
1,Summoners War,2426960,A global hit RPG that captured the Summoners a...,[Com2uS],[Com2uS],"[RPG, Strategy, Free To Play]","7 Jan, 2024",False,None,True
2,DOOM 3,9050,A massive demonic invasion has overwhelmed the...,[id Software],[id Software],[Action],"3 Aug, 2007",False,Violence\r\nLanguage,True
3,PAYDAY 3,1272080,PAYDAY 3 is the much anticipated sequel to one...,[Starbreeze Studios],[Deep Silver],"[Action, Adventure, RPG]","21 Sep, 2023",False,PAYDAY 3 contains repeated displays of gun vio...,True
4,Cult of the Lamb,1313140,Start your own cult in a land of false prophet...,[Massive Monster],[Devolver Digital],"[Action, Adventure, Indie, Strategy]","11 Aug, 2022",False,Cartoon Violence,True
...,...,...,...,...,...,...,...,...,...,...
145,Overcooked! All You Can Eat,1243830,"Experience all the Onion Kingdom has to offer,...","[Team17 Digital, Ghost Town Games]",[Team17 Digital],"[Casual, Indie, Simulation, Strategy]","23 Mar, 2021",False,None,True
146,NieR:Automata™,524220,"NieR: Automata tells the story of androids 2B,...","[Square Enix, PlatinumGames Inc.]",[Square Enix],"[Action, RPG]","17 Mar, 2017",False,None,True
147,Until Then,1574820,"A fateful meeting sets off a chain reaction, u...",[Polychroma Games],[Maximum Entertainment],"[Adventure, Casual, Indie]","25 Jun, 2024",False,None,True
148,Yu-Gi-Oh! Duel Links,601510,¡Enfréntate a duelistas de todo el mundo con Y...,[KONAMI],[KONAMI],"[Estrategia, Free to Play]",16 NOV 2017,False,None,True


In [39]:
time.sleep(300)
games_id_details_3 = pd.DataFrame()
for _,game in games_id[300:450].iterrows():
    dict = appdata_req(game)
    games_id_details_3 = pd.concat([games_id_details_3, pd.DataFrame(dict)], ignore_index = True)

games_id_details_3

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Contraband Police,756800,Take over the duties of a border guard inspect...,[Crazy Rocks],[PlayWay S.A.],"[Action, Adventure, Simulation]","8 Mar, 2023",False,This game may contain content that is not appr...,True
1,KINGDOM HEARTS -HD 1.5+2.5 ReMIX-,2552430,KINGDOM HEARTS -HD 1.5+2.5 ReMIX- is an HD rem...,[Square Enix],[Square Enix],"[Action, Adventure, RPG]","13 Jun, 2024",False,None,True
2,Horizon Zero Dawn™ Complete Edition,1151640,Experience Aloy’s legendary quest to unravel t...,[Guerrilla],[PlayStation Publishing LLC],"[Action, Adventure, RPG]","7 Aug, 2020",False,None,True
3,Detroit: Become Human,1222140,Detroit: Become Human puts the destiny of both...,[Quantic Dream],[Quantic Dream],"[Action, Adventure]","18 Jun, 2020",False,None,True
4,Grand Theft Auto IV: The Complete Edition,12210,"Niko Bellic, Johnny Klebitz and Luis Lopez all...","[Rockstar North, Rockstar Toronto]",[Rockstar Games],"[Action, Adventure]","24 Mar, 2020",False,None,True
...,...,...,...,...,...,...,...,...,...,...
145,Fabledom,1651560,"Set in a wholesome fairytale world, Fabledom i...",[Grenaa Games],"[Dear Villagers, Doyoyo Games]","[Casual, Simulation, Strategy]","13 May, 2024",False,None,True
146,Fran Bow,362680,Fran Bow is a creepy adventure game that tells...,[Killmonday Games AB],[Killmonday Games AB],"[Adventure, Indie]","27 Aug, 2015",False,None,True
147,Furnish Master,2004080,Furnish Master is a relaxing game about furnis...,[Alex Blintsov],[Alex Blintsov],"[Casual, Indie, Simulation, Early Access]","19 Feb, 2024",False,None,True
148,Brotato,1942280,Brotato is a top-down arena shooter roguelite ...,[Blobfish],[Blobfish],"[Action, Casual, Indie, RPG]","23 Jun, 2023",False,None,True


In [116]:
time.sleep(300)
games_id_details_4 = pd.DataFrame()
for _,game in games_id[450:600].iterrows():
    dict = appdata_req(game)
    games_id_details_4 = pd.concat([games_id_details_4, pd.DataFrame(dict)], ignore_index = True)

games_id_details_4

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Diplomacy is Not an Option,1272320,Diplomacy is Not an Option is a strategy game ...,[Door 407],[Door 407],"[Indie, Strategy, Early Access]","9 Feb, 2022",False,Massive cartoon violence.,True
1,Rise of Nations: Extended Edition,287450,Rise of Nations is back! Play the updated clas...,"[SkyBox Labs, Big Huge Games]",[Xbox Game Studios],"[Simulation, Strategy]","12 Jun, 2014",False,None,True
2,MotoGP™24,2581700,"Rev up your engines, MotoGP™24 is back! Dive i...",[Milestone S.r.l.],[Milestone S.r.l.],"[Racing, Sports]","2 May, 2024",False,None,True
3,Spirit City: Lofi Sessions,2113850,Spirit City: Lofi Sessions is a gamified focus...,[Mooncube Games],[Mooncube Games],"[Casual, Indie, Simulation]","8 Apr, 2024",False,None,True
4,Need for Speed™ Hot Pursuit Remastered,1328660,Feel the thrill of the chase and the rush of e...,[Stellar Entertainment Limited],[Electronic Arts],"[Action, Racing]","6 Nov, 2020",False,None,True
...,...,...,...,...,...,...,...,...,...,...
145,Steelrising,1283400,"Paris, 1789. The French Revolution has been su...",[Spiders],[Nacon],"[Action, Adventure, RPG]","8 Sep, 2022",False,The game features violence during gameplay.,True
146,Ship Graveyard Simulator 2,2201940,Dust off your torch and hammer - it's time to ...,"[Games Incubator, Hypnotic Ants]","[Games Incubator, PlayWay S.A.]","[Action, Adventure, Casual, Indie, Simulation]","16 Aug, 2023",False,None,True
147,SONIC X SHADOW GENERATIONS,2513280,Shadow the Hedgehog is back with Classic and M...,[Sonic Team],[SEGA],[Action],"24 Oct, 2024",True,None,True
148,Jujutsu Kaisen Cursed Clash,1877020,Master the Jujutsu of your favorite Sorcerers ...,[Byking Inc.],[Bandai Namco Entertainment Inc.],[Action],"1 Feb, 2024",False,None,True


In [48]:
time.sleep(300)
games_id_details_5 = pd.DataFrame()
for _,game in games_id[600:750].iterrows():
    dict = appdata_req(game)
    games_id_details_5 = pd.concat([games_id_details_5, pd.DataFrame(dict)], ignore_index = True)

games_id_details_5

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Gestalt: Steam & Cinder,1231990,"Inspired by 16 and 32-bit classics, Gestalt: S...",[Metamorphosis Games],[Fireshine Games],"[Action, Adventure, Indie, RPG]","16 Jul, 2024",False,None,True
1,Escape the Backrooms,1943950,Escape the Backrooms is a 1-4 player co-op hor...,[Fancy Games],[Fancy Games],"[Indie, Early Access]","11 Aug, 2022",False,None,True
2,Scrap Mechanic,387990,Enter the creative paradise of Scrap Mechanic!...,[Axolot Games],[Axolot Games],"[Action, Adventure, Indie, Simulation, Early A...","19 Jan, 2016",False,None,True
3,Ogu and the Secret Forest,1985960,Explore the wonderful world with baby Ogu! 'Og...,"[Moonlab Studio, Sinkhole Studio]",[ Sinkhole Studio],[Adventure],"29 Jul, 2024",False,None,True
4,Magicka 2,238370,The world’s most irreverent co-op action adven...,[Pieces Interactive],[Paradox Interactive],"[Action, Adventure]","26 May, 2015",False,None,True
...,...,...,...,...,...,...,...,...,...,...
145,XCOM: Enemy Unknown,200510,The XCOM: Enemy Unknown - Slingshot Pack is No...,"[Firaxis Games, Feral Interactive (Linux)]","[2K, Feral Interactive (Linux)]",[Strategy],"10 Oct, 2012",False,None,True
146,Gladiators: Survival in Rome,2295520,Gladiators: Survival in Rome is an Action RPG ...,[Colossi Games],[Colossi Games],"[Action, Adventure, RPG, Free To Play, Early A...","13 Apr, 2023",False,None,True
147,Viewfinder,1382070,"Challenge perception, redefine reality, and re...",[Sad Owl Studios],[Thunderful Publishing],"[Adventure, Indie]","18 Jul, 2023",False,None,True
148,Trove,304050,"Grab your friends, hone your blades, and set o...",[gamigo US Inc.],[gamigo US Inc.],"[Action, Adventure, Casual, Massively Multipla...","9 Jul, 2015",False,None,True


In [49]:
time.sleep(300)
games_id_details_6 = pd.DataFrame()
for _,game in games_id[750:900].iterrows():
    dict = appdata_req(game)
    games_id_details_6 = pd.concat([games_id_details_6, pd.DataFrame(dict)], ignore_index = True)

games_id_details_6

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Kebab Chefs! - Restaurant Simulator,1001270,"Kebab Chefs! is a Co-Op, cooking and restauran...",[Biotech Gameworks],[Biotech Gameworks],"[Simulation, Early Access]","19 Jan, 2024",False,None,True
1,Tiny Terry's Turbo Trip,2238040,Terry wants to launch himself into space using...,[snekflat],[Super Rare Originals],"[Adventure, Casual, Indie, RPG]","30 May, 2024",False,None,True
2,Mini Metro,287980,Mini Metro is a strategy simulation game about...,[Dinosaur Polo Club],[Dinosaur Polo Club],"[Indie, Simulation, Strategy]","6 Nov, 2015",False,None,True
3,Big Farm Story,1329510,Build your dream farm with your friends &amp; ...,[Goodgame Studios],[Goodgame Studios],"[Adventure, Casual, RPG, Simulation]","2 Aug, 2021",False,None,True
4,Anno 1800,916440,Anno 1800™ – Lead the Industrial Revolution! W...,[Ubisoft Mainz],[Ubisoft],"[Simulation, Strategy]",Apr 2019,False,None,True
...,...,...,...,...,...,...,...,...,...,...
145,The Unfinished Swan,1206430,From the creators of What Remains of Edith Fin...,[Giant Sparrow],[Annapurna Interactive],"[Adventure, Indie]","10 Sep, 2020",False,None,True
146,Summer in Mara,962580,Take care of your own island and sail with you...,[Chibig],[Chibig],"[Adventure, Casual, Indie, RPG, Simulation]","16 Jun, 2020",False,None,True
147,Forgive Me Father,1590910,Forgive Me Father is a dark retro horror FPS s...,[Byte Barrel],[Fulqrum Publishing],"[Action, Indie]","7 Apr, 2022",False,This game is stylized Lovecraftian old-school ...,True
148,RWBY: Grimm Eclipse,418340,"RWBY: GRIMM ECLIPSE is a 4 player, online co-o...",[Rooster Teeth Games],[Rooster Teeth Games],"[Action, Adventure, Indie]","5 Jul, 2016",False,None,True


In [50]:
time.sleep(300)
games_id_details_7 = pd.DataFrame()
for _,game in games_id[900:1050].iterrows():
    dict = appdata_req(game)
    games_id_details_7 = pd.concat([games_id_details_7, pd.DataFrame(dict)], ignore_index = True)

games_id_details_7

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,I Am Fish,1472560,"I Am Fish is a charming, physics-based adventu...",[Bossa Studios],[Curve Games],"[Adventure, Simulation]","16 Sep, 2021",False,None,True
1,Bee Simulator,914750,Live the big adventure of a small bee! Explore...,[VARSAV Game Studios],[Nacon],"[Adventure, Casual, Simulation]","17 Nov, 2020",False,None,True
2,Pascal's Wager: Definitive Edition,1456650,Pascal's Wager: Definitive Edition is a challe...,[TipsWorks],[Giant Global],"[Action, Adventure, RPG]","11 Mar, 2021",False,None,True
3,Trackmania,2225070,Welcome to the Club! Grab the wheel and play T...,[Nadeo],[Ubisoft],[Racing],"2 Feb, 2023",False,None,True
4,CONSCRIPT,1286990,"During the First World War, a lone French sold...","[Jordan Mochi, Catchweight Studio]",[Team17],"[Action, Adventure, Indie]","23 Jul, 2024",False,This game contains scenes of explicit violence...,True
...,...,...,...,...,...,...,...,...,...,...
145,TerraScape,2290000,Build gorgeous kingdoms on floating islands by...,[Bitfall Studios],"[Stray Fawn Publishing, INSTINCT3]","[Casual, Indie, Simulation, Strategy]","17 Jul, 2024",False,None,True
146,Simple CEO,2704510,"Become a titan of industry! Simple to learn, d...",[Madowl Games],[Madowl Games],"[Casual, Indie, Strategy]","5 Jan, 2024",False,None,True
147,Little Inferno,221260,"Throw your toys into your fire, and play with ...",[Tomorrow Corporation],[Tomorrow Corporation],"[Adventure, Casual, Indie]","19 Nov, 2012",False,None,True
148,Abathor,2295610,Abathor is a challenging 2D action platformer ...,[Pow Pixel Games],[JanduSoft],"[Action, Adventure, Indie]","25 Jul, 2024",False,None,True


In [51]:
time.sleep(300)
games_id_details_8 = pd.DataFrame()
for _,game in games_id[1050:1200].iterrows():
    dict = appdata_req(game)
    games_id_details_8 = pd.concat([games_id_details_8, pd.DataFrame(dict)], ignore_index = True)

games_id_details_8

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Plague Inc: Evolved,246620,Plague Inc: Evolved is a unique mix of high st...,[Ndemic Creations],[Ndemic Creations],"[Casual, Indie, Simulation, Strategy]","18 Feb, 2016",False,None,True
1,Loop Hero,1282730,The Lich has thrown the world into a timeless ...,[Four Quarters],[Devolver Digital],"[Indie, RPG, Strategy]","4 Mar, 2021",False,None,True
2,I Wani Hug that Gator!,1895350,I Wani Hug that Gator! Is a choice-driven narr...,[Cavemanon],[Cavemanon],"[Casual, Indie]","14 Feb, 2024",False,"Implicit reference to drug use, including impl...",True
3,Them and Us,915600,"Them and Us is a classic, survival horror game...","[TendoGames Co., Ltd.]","[TendoGames Co., Ltd. , 方块游戏(CubeGame)]","[Action, Adventure, Indie]","28 Sep, 2021",False,This Game may contain content not appropriate ...,True
4,Tower Hunter: Erza's Trial,844850,Tower Hunter: Erza’s Trial contains core eleme...,[IceSitruuna],[IceSitruuna],"[Action, Adventure, Casual, Indie, RPG]","12 Oct, 2019",False,None,True
...,...,...,...,...,...,...,...,...,...,...
145,Potion Tycoon,1485690,Potion Tycoon is a management simulator with a...,[Snowhound Games],[Daedalic Entertainment],"[Simulation, Strategy, Early Access]","13 Mar, 2023",False,None,True
146,LostWinds 2: Winter of the Melodias,447800,"With amazing new abilities, re-visit a land fu...",[Frontier Developments],[Frontier Developments],[Adventure],"24 Mar, 2016",False,None,True
147,Capcom Arcade 2nd Stadium,1755910,Play more of your favorite arcade hits from ye...,"[CAPCOM Co., Ltd.]","[CAPCOM Co., Ltd.]","[Action, Free To Play]","21 Jul, 2022",False,None,True
148,Sonic Adventure DX,71250,An ancient evil lurking within the Master Emer...,[SEGA],[SEGA],None,"4 Mar, 2011",False,None,True


In [52]:
time.sleep(300)
games_id_details_9 = pd.DataFrame()
for _,game in games_id[1200:1350].iterrows():
    dict = appdata_req(game)
    games_id_details_9 = pd.concat([games_id_details_9, pd.DataFrame(dict)], ignore_index = True)

games_id_details_9

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Grow Home,323320,Experience the freedom of unbounded climbing a...,"[Reflections, a Ubisoft Studio]",[Ubisoft],"[Adventure, Casual, Indie]","4 Feb, 2015",False,None,True
1,Mad Tower Tycoon,910880,Design your own skyscraper with more than 100 ...,[Eggcode],[Eggcode],"[Indie, Simulation, Strategy]","14 Jan, 2020",False,None,True
2,Potion Craft: Alchemist Simulator,1210320,Potion Craft is an alchemist simulator where y...,[niceplay games],[tinyBuild],"[Indie, Simulation]","13 Dec, 2022",False,None,True
3,ArcaniA: Fall of Setarrif,65610,Face the last challenge - determine destiny! A...,[Spellbound],[THQ Nordic],[RPG],"24 Oct, 2011",False,None,True
4,Heavenly Bodies,1138850,Wrangle the hands and arms of a 1970s space co...,[2pt Interactive],[2pt Interactive],"[Action, Adventure, Indie, Simulation]","7 Dec, 2021",False,None,True
...,...,...,...,...,...,...,...,...,...,...
145,Rogue Warrior,22319,None,None,None,None,None,None,None,False
146,ARCADE GAME SERIES: PAC-MAN,394160,"First released in arcades in 1980, the masterp...",[BANDAI NAMCO Studios Inc.],[BANDAI NAMCO Entertainment],[Action],"19 Apr, 2016",False,None,True
147,Full Spectrum Warrior,4520,The U.S. Army is the most powerful ground forc...,[Pandemic Studios],[THQ Nordic],"[Action, Strategy]","17 Jul, 2007",False,None,True
148,ibb & obb,95400,ibb &amp; obb is a two player cooperative game...,"[Sparpweed, Codeglue]",[Sparpweed],"[Action, Adventure, Indie]","26 May, 2014",False,None,True


In [53]:
time.sleep(300)
games_id_details_10 = pd.DataFrame()
for _,game in games_id[1350:].iterrows():
    dict = appdata_req(game)
    games_id_details_10 = pd.concat([games_id_details_10, pd.DataFrame(dict)], ignore_index = True)

games_id_details_10

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,Demetrios - The BIG Cynical Adventure,451570,The Journey of an Idiot. 8 to 12 hours of fun!,[COWCAT],[COWCAT],"[Adventure, Casual, Indie]","31 May, 2016",False,None,True
1,Barro F22,1958600,The new gen for the F version!,[SC Jogos],[SC Jogos],"[Casual, Indie, Racing]","25 Apr, 2022",False,None,True
2,Barro 22,1797610,Jump!,[SC Jogos],[SC Jogos],"[Casual, Indie, Racing]","5 Nov, 2021",False,None,True
3,STASIS,380150,"STASIS The isometric, point-and-click, sci-fi ...",[THE BROTHERHOOD],[THE BROTHERHOOD],"[Adventure, Indie]","31 Aug, 2015",False,None,True
4,The White Door,1145960,Enter The White Door and discover Rusty Lake’s...,[Rusty Lake],[Second Maze],"[Adventure, Indie]","9 Jan, 2020",False,None,True
...,...,...,...,...,...,...,...,...,...,...
108,Farming Tractor Simulator: Big Farm,2315010,"In the farm game, you can drive a 3d tractor a...","[Yusuf Islam Seyhan, Samet Acar]",[Inspector Studios],"[Adventure, Casual, Indie, RPG, Simulation]","23 Feb, 2023",False,None,True
109,Possessed,607210,Possessed is a fast-paced story driven surviva...,[Volens Nolens Games],[Volens Nolens Games],"[Action, Adventure]","17 May, 2017",False,None,True
110,Fighters Unleashed,566560,Fighters Unleashed is an old school Beat'em up...,[Volens Nolens Games],[Volens Nolens Games],"[Action, Adventure, Indie]","27 Dec, 2016",False,None,True
111,Fireside,1722700,Fireside is an uplifting narrative experience ...,[Emergo Entertainment],[Nordcurrent Labs],"[Adventure, Indie, Strategy]","4 Jun, 2024",False,None,True


In [117]:
games_id_details = pd.DataFrame()
particiones = [games_id_details_1, games_id_details_2, games_id_details_3, games_id_details_4, games_id_details_5, games_id_details_6,
              games_id_details_7, games_id_details_8, games_id_details_9, games_id_details_10]
games_id_details = pd.concat(particiones, ignore_index = True)

In [96]:
games_id_details

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,PUBG: BATTLEGROUNDS,578080,Play PUBG: BATTLEGROUNDS for free. Land on str...,"[KRAFTON, Inc.]","[KRAFTON, Inc.]","[Action, Adventure, Massively Multiplayer, Fre...","21 Dec, 2017",False,None,True
1,Counter-Strike 2,730,"For over two decades, Counter-Strike has offer...",[Valve],[Valve],"[Action, Free To Play]","21 Aug, 2012",False,Includes intense violence and blood.,True
2,Call of Duty®: Black Ops 6,2933620,Call of Duty®: Black Ops 6 is signature Black ...,"[Treyarch, Raven Software, Beenox, High Moon S...",[Activision],[Action],"24 Oct, 2024",True,None,True
3,God of War,1593500,His vengeance against the Gods of Olympus year...,"[Santa Monica Studio, Jetpack Interactive]",[PlayStation Publishing LLC],"[Action, Adventure, RPG]","14 Jan, 2022",False,Gameplay consists of frequent combat scenarios...,True
4,EA SPORTS FC™ 25,3102940,None,None,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...
1458,Farming Tractor Simulator: Big Farm,2315010,"In the farm game, you can drive a 3d tractor a...","[Yusuf Islam Seyhan, Samet Acar]",[Inspector Studios],"[Adventure, Casual, Indie, RPG, Simulation]","23 Feb, 2023",False,None,True
1459,Possessed,607210,Possessed is a fast-paced story driven surviva...,[Volens Nolens Games],[Volens Nolens Games],"[Action, Adventure]","17 May, 2017",False,None,True
1460,Fighters Unleashed,566560,Fighters Unleashed is an old school Beat'em up...,[Volens Nolens Games],[Volens Nolens Games],"[Action, Adventure, Indie]","27 Dec, 2016",False,None,True
1461,Fireside,1722700,Fireside is an uplifting narrative experience ...,[Emergo Entertainment],[Nordcurrent Labs],"[Adventure, Indie, Strategy]","4 Jun, 2024",False,None,True


In [118]:
games_id_details.to_csv('games_id_details.csv', index=False)

# PASO 3

Ejecutar desde aqui si ya se ha hecho bien la construccion del paso1 y 2


In [121]:
games_data = pd.read_csv('games_id_details.csv') 

In [122]:
games_data = games_data[games_data['Success'] == True]

In [123]:
games_data = games_data[games_data['Soon'] == False]

In [124]:
games_data = games_data.drop_duplicates(['Name', 'AppID'])

In [125]:
games_data['Release_Date'] = pd.to_datetime(games_data['Release_Date'], format='mixed')

In [126]:
games_data

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
0,PUBG: BATTLEGROUNDS,578080,Play PUBG: BATTLEGROUNDS for free. Land on str...,"['KRAFTON, Inc.']","['KRAFTON, Inc.']","['Action', 'Adventure', 'Massively Multiplayer...",2017-12-21,False,NaN,True
1,Counter-Strike 2,730,"For over two decades, Counter-Strike has offer...",['Valve'],['Valve'],"['Action', 'Free To Play']",2012-08-21,False,Includes intense violence and blood.,True
3,God of War,1593500,His vengeance against the Gods of Olympus year...,"['Santa Monica Studio', 'Jetpack Interactive']",['PlayStation Publishing LLC'],"['Action', 'Adventure', 'RPG']",2022-01-14,False,Gameplay consists of frequent combat scenarios...,True
5,Borderlands 3,397540,"The original shooter-looter returns, packing b...",['Gearbox Software'],['2K'],"['Action', 'RPG']",2020-03-13,False,This Game may contain content not appropriate ...,True
7,Prince of Persia The Lost Crown,2751000,Dash into a stylish and thrilling action-adven...,['Ubisoft Montpellier'],['Ubisoft'],"['Action', 'Adventure']",2024-08-08,False,NaN,True
...,...,...,...,...,...,...,...,...,...,...
1458,Farming Tractor Simulator: Big Farm,2315010,"In the farm game, you can drive a 3d tractor a...","['Yusuf Islam Seyhan', 'Samet Acar']",['Inspector Studios'],"['Adventure', 'Casual', 'Indie', 'RPG', 'Simul...",2023-02-23,False,NaN,True
1459,Possessed,607210,Possessed is a fast-paced story driven surviva...,['Volens Nolens Games'],['Volens Nolens Games'],"['Action', 'Adventure']",2017-05-17,False,NaN,True
1460,Fighters Unleashed,566560,Fighters Unleashed is an old school Beat'em up...,['Volens Nolens Games'],['Volens Nolens Games'],"['Action', 'Adventure', 'Indie']",2016-12-27,False,NaN,True
1461,Fireside,1722700,Fireside is an uplifting narrative experience ...,['Emergo Entertainment'],['Nordcurrent Labs'],"['Adventure', 'Indie', 'Strategy']",2024-06-04,False,NaN,True


In [129]:
# Estos que quedan se miran a mano
games_data[games_data.duplicated('Name', keep = False)]

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success
13,Rust,980030,Heather and her best friend Morgan make the mi...,NaN,['Wild Eye Releasing'],['Movie'],2018-11-14,False,There is horror related violence throughout th...,True
14,Rust,252490,The only aim in Rust is to survive. Everything...,['Facepunch Studios'],['Facepunch Studios'],"['Action', 'Adventure', 'Indie', 'Massively Mu...",2018-02-08,False,Contains violence and caveman themed nudity.\r...,True
110,Prey,480490,"In Prey, you awaken aboard Talos I, a space st...",['Arkane Studios'],['Bethesda Softworks'],"['Action', 'Adventure']",2017-05-04,False,NaN,True
111,Prey,3970,Our sale on Prey was so popular we ran out of ...,['Humanhead Studios'],['2K Games'],['Action'],2006-11-30,False,NaN,True
889,Last Stop,2327980,In Last Stop you awaken alone in a train that’...,['Tainted Games'],['Tainted Games'],"['Action', 'Adventure', 'Indie']",2023-03-13,False,The game contains images of scenes of violence...,True
890,Last Stop,1122690,Last Stop is a single-player third-person adve...,['Variable State'],['Annapurna Interactive'],"['Adventure', 'Indie']",2021-07-22,False,NaN,True
970,Let Me Out,2025770,"John, a 30-year-old detective experiences para...",['THE_HUNTER_MX'],['EDMO STUDIO'],"['Adventure', 'Indie', 'Early Access']",2022-09-30,False,NaN,True
971,Let Me Out,1269520,"Play as Alvin, a 10-year-old indigo child, in ...",['4Happy Studio'],"['4Happy Studio', 'PID Games']","['Action', 'Adventure', 'Indie']",2024-08-08,False,NaN,True
1078,Eternal Return,1049590,"Eternal Return is a free-to-play, 2.5D perspec...",['Nimble Neuron'],"['Nimble Neuron', 'SapStaR Games']","['Indie', 'Strategy', 'Free To Play']",2023-07-19,False,NaN,True
1079,Eternal Return,528010,Face merciless gameplay to explore a cursed ca...,['Pathless Games'],['Pathless Games'],"['Action', 'Adventure', 'Indie']",2017-02-23,False,NaN,True


In [132]:
games_data = games_data.drop(13)

In [139]:
# Estos que quedan se miran a mano, eliminamos el antiguo
games_data = games_data.drop(games_data[games_data.duplicated('Name')].sort_values('Release_Date', ascending = False).index)

In [140]:
games_data[games_data.duplicated('Name', keep = False)]

,Name,AppID,Description,Devs,Publishers,Genres,Release_Date,Soon,Notes,Success


In [142]:
games_data.to_csv('games_data.csv', index=False)

#Concluimos la limpieza de la tabla de datos de los juegos

# PASO 4

Construccion de tabla de comentarios, tambien recuperados de la API de Steam https://partner.steamgames.com/doc/store/getreviews

In [179]:
# Vamos como antes primero con un ejemplo 
review_appid_test = 730
parametros = {
        'json':1,
        'language': 'english',
        'cursor': '*',                                  
        'num_per_page': 100,
        'filter': 'updated'
    }

game_reviews_url_test = f'https://store.steampowered.com/appreviews/{review_appid_test}'
req_reviews_test = requests.get(
    game_reviews_url_test,
    params=parametros
)

game_reviews_test = req_reviews_test.json()
game_reviews_test

{'success': 1,
 'query_summary': {'num_reviews': 100,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 1083019,
  'total_negative': 162308,
  'total_reviews': 1245327},
 'reviews': [{'recommendationid': '171885676',
   'author': {'steamid': '76561199756253130',
    'num_games_owned': 0,
    'num_reviews': 1,
    'playtime_forever': 493,
    'playtime_last_two_weeks': 493,
    'playtime_at_review': 490,
    'last_played': 1723370636},
   'language': 'english',
   'review': 'nice',
   'timestamp_created': 1723370446,
   'timestamp_updated': 1723370446,
   'voted_up': True,
   'votes_up': 0,
   'votes_funny': 0,
   'weighted_vote_score': 0,
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': False,
   'hidden_in_steam_china': True,
   'steam_china_location': ''},
  {'recommendationid': '171885589',
   'author': {'steamid': '76561199654842361',
    'num_games_owned': 1,
    'num_reviews': 1,
    

In [180]:
lista_reviews_test = []

if game_reviews_test.get('reviews',None) != None:
    for x in game_reviews_test.get('reviews',None):
        review_test = {}
        review_test['AppID'] = review_appid_test
        review_test['ReviewID'] = x.get('recommendationid',None)
        review_test['UserID'] = x.get('author',None).get('steamid',None)
        review_test['Language'] = x.get('language',None)
        review_test['Review'] = x.get('review',None)
        review_test['Positive'] = x.get('voted_up',None)
        review_test['Bought'] = x.get('steam_purchase',None)
        review_test['Free'] = x.get('received_for_free',None)

        lista_reviews_test.append(review_test)

lista_reviews_test
#Esto pinta las reviews de un juego

game_review_df_test = pd.DataFrame(lista_reviews_test)
game_review_df_test
#Y esto nos da el dataframe

,AppID,ReviewID,UserID,Language,Review,Positive,Bought,Free
0,730,171885676,76561199756253130,english,nice,True,True,False
1,730,171885589,76561199654842361,english,dog china cheater,False,True,False
2,730,32152901,76561198197015899,english,Погана гра. Не раджу,False,True,False
3,730,171885227,76561199485748245,english,I lost my all skins bc i was hacked i want the...,True,True,False
4,730,171884997,76561199535905428,english,It's ight.,True,True,False
...,...,...,...,...,...,...,...,...
95,730,171864629,76561198866116540,english,too many cheaters. VAC or VALVE dont do their ...,False,True,False
96,730,85792702,76561198207960243,english,game is terrible and not fun and gay,False,True,False
97,730,171864470,76561199224679631,english,This game makes me the happiest man alive,True,True,False
98,730,136780462,76561199085693559,english,pulled a knife after only spending $20 10/10 w...,False,True,False


In [218]:
review_appid_test_2 = 2349820

game_reviews_url_test_2 = f'https://store.steampowered.com/appreviews/{review_appid_test_2}'
req_reviews_test_2 = requests.get(
    game_reviews_url_test_2,
    params={
        'json':1,
        'language': 'english',
        'cursor': '*',                                  
        'num_per_page': 100,
        'filter': 'recent',
        'purchase_type': 'non_steam_purchase' #################3
    }
)

game_reviews_test_2 = req_reviews_test_2.json()
game_reviews_test_2

{'success': 1,
 'query_summary': {'num_reviews': 100,
  'review_score': 6,
  'review_score_desc': 'Mostly Positive',
  'total_positive': 1018,
  'total_negative': 344,
  'total_reviews': 1362},
 'reviews': [{'recommendationid': '171863427',
   'author': {'steamid': '76561199141636627',
    'num_games_owned': 0,
    'num_reviews': 5,
    'playtime_forever': 1833,
    'playtime_last_two_weeks': 1833,
    'playtime_at_review': 1695,
    'last_played': 1723374546},
   'language': 'english',
   'review': 'Its good game but PVP eh.',
   'timestamp_created': 1723337634,
   'timestamp_updated': 1723337634,
   'voted_up': True,
   'votes_up': 1,
   'votes_funny': 0,
   'weighted_vote_score': '0.504504501819610596',
   'comment_count': 0,
   'steam_purchase': False,
   'received_for_free': False,
   'written_during_early_access': False,
   'hidden_in_steam_china': True,
   'steam_china_location': ''},
  {'recommendationid': '171851995',
   'author': {'steamid': '76561198348480189',
    'num_game

In [220]:
 #Funcion para recoger las reviews del json y usarlos en un DF
def reviews_req(game,params):
    request_reviews = requests.get(f"https://store.steampowered.com/appreviews/{game['AppID']}", params = params)  
    lista_reviews = []
    if request_reviews.status_code == 200:
        game_reviews = request_reviews.json()
    
        if game_reviews.get('reviews',None) != None:
            for x in game_reviews.get('reviews',None):
                review = {}
                review['AppID'] = game['AppID']
                review['ReviewID'] = x.get('recommendationid',None)
                review['UserID'] = x.get('author',None).get('steamid',None)
                review['Language'] = x.get('language',None)
                review['Review'] = x.get('review',None)
                review['Positive'] = x.get('voted_up',None)
                review['Bought'] = x.get('steam_purchase',None)
                review['Free'] = x.get('received_for_free',None)
                
                lista_reviews.append(review)

    
    if game_reviews.get('query_summary',None).get('num_reviews',None) == 0: # Son aquellos free to play que no se consideran comprados en steam
        params['purchase_type'] = 'non_steam_purchase' #Empezamos de nuevo con esta condicion extra
        request_reviews = requests.get(f"https://store.steampowered.com/appreviews/{game['AppID']}", params = params)  
        
        if request_reviews.status_code == 200:
            game_reviews = request_reviews.json()
        
            if game_reviews.get('reviews',None) != None:
                for x in game_reviews.get('reviews',None):
                    review = {}
                    review['AppID'] = game['AppID']
                    review['ReviewID'] = x.get('recommendationid',None)
                    review['UserID'] = x.get('author',None).get('steamid',None)
                    review['Language'] = x.get('language',None)
                    review['Review'] = x.get('review',None)
                    review['Positive'] = x.get('voted_up',None)
                    review['Bought'] = x.get('steam_purchase',None)
                    review['Free'] = x.get('received_for_free',None)
                    
                    lista_reviews.append(review)
                
    game_review_df = pd.DataFrame(lista_reviews)
    
    return game_review_df

In [229]:
#Recogemos un dataframe en ingles:
parametros = {
        'json':1,
        'language': 'english',
        'cursor': '*',                                  
        'num_per_page': 100,
        'filter': 'updated'
    }
games_reviews_df = pd.DataFrame()

for particion in range(10):
    time.sleep(300) #esperamos 5 mins entre particiones por la limitacion del api de steam
    for _,game in games_data[particion*150:(particion+1)*150].iterrows():
        games_reviews_df = pd.concat([games_reviews_df, reviews_req(game, parametros)], ignore_index = True)

games_reviews_df 

,AppID,ReviewID,UserID,Language,Review,Positive,Bought,Free
0,578080,171886441,76561198150195741,english,don' recommend,False,True,False
1,578080,137852244,76561198871670200,english,"The past couple of months, PUBG has been impro...",False,True,False
2,578080,171878328,76561198105543998,english,"Great game, highly recommend",True,True,False
3,578080,171877513,76561198011887521,english,Classic,True,True,False
4,578080,79647082,76561198390827775,english,돈 주고 뽑은 애스턴마틴만 타면 대가리 다터지는 병신같은게임,False,True,False
...,...,...,...,...,...,...,...,...
108119,1972440,155655678,76561198039515791,english,This game just isn't fun after the first hour....,False,False,False
108120,1972440,155643442,76561197995272591,english,I didn't think I would enjoy this little gem a...,True,False,False
108121,1972440,155512889,76561198147486516,english,help I'm being held at gun point to write a go,True,False,False
108122,1972440,155483819,76561197971306794,english,5/10 - You and your opponent get a perk after ...,False,False,False


In [230]:
games_reviews_df.to_csv('games_reviews_df.csv', index=False)

In [231]:
#Recogemos un dataframe en español:
parametros_esp = {
        'json':1,
        'language': 'spanish',
        'cursor': '*',                                  
        'num_per_page': 100,
        'filter': 'updated'
    }
games_reviews_df_esp = pd.DataFrame()

for particion in range(10):
    time.sleep(300) #esperamos 5 mins entre particiones por la limitacion del api de steam
    for _,game in games_data[particion*150:(particion+1)*150].iterrows():
        games_reviews_df_esp = pd.concat([games_reviews_df_esp, reviews_req(game, parametros_esp)], ignore_index = True)

games_reviews_df_esp

,AppID,ReviewID,UserID,Language,Review,Positive,Bought,Free
0,578080,65431415,76561199004698471,spanish,"es buen juego, muy entretenido la verdad y es ...",True,True,False
1,578080,171877626,76561198200965553,spanish,lo +,True,True,False
2,578080,171871718,76561199157928123,spanish,el juego era bueno antes ahorita es una porque...,False,True,False
3,578080,83524718,76561198289440199,spanish,"Malo, no vale la pena",False,True,False
4,578080,171861213,76561198159686009,spanish,"Definitivamente una droga, te vuelve adicto. I...",True,True,False
...,...,...,...,...,...,...,...,...
69513,1972440,115596533,76561198268658395,spanish,"Exageradamente divertido, al igual que el ajed...",True,False,False
69514,1972440,115583730,76561198986177306,spanish,Ajedrez pero eres el rey contra todos y tienes...,True,False,False
69515,1972440,115535475,76561198080626510,spanish,Amazing idea!,True,False,False
69516,1972440,115514818,76561198118861600,spanish,Muy divertido y el precio es justo para el con...,True,False,False


In [232]:
games_reviews_df_esp.to_csv('games_reviews_df_esp.csv', index=False)